In [168]:
import sys
sys.path.insert(1, '/scratch/cinthiasouza/mv-text-summarizer')

import glob, os
import pandas as pd
import json
import spacy
import nltk
import numpy as np
import json
#import smogn
import seaborn as sns
import pickle

from bs4 import BeautifulSoup
from pysbd.utils import PySBDFactory
import math

from sumeval.metrics.rouge import RougeCalculator
rouge = RougeCalculator(stopwords=True, lang="en")
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from timeit import default_timer as timer 
from imblearn.over_sampling import SMOTE
from sklearn.metrics import matthews_corrcoef
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
#from tensorflow.keras.models import model_from_json

#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('punkt')
#nltk.download('stopwords')

#!python -m spacy download en_core_web_sm
#nlp_sm = spacy.load('en_core_web_sm')

import en_core_web_sm
nlp_md = en_core_web_sm.load()

import en_core_web_md
nlp_md = en_core_web_md.load()
#!python -m spacy download en_core_web_md
#nlp_md = spacy.load('en_core_web_md')

import warnings
warnings.filterwarnings("ignore")

path_base = "/scratch/cinthiasouza/mv-text-summarizer"


In [144]:
def main(name_section, initial_batch, embed_dim, batch_len, path_base, verbose):
    
    features_columns = ['sentences', 'sentences_text', 'sentences_lex', 'sentences_lsa',
                        'count_one_gram', 'count_two_gram', 'count_three_gram', 
                        'count_article_keywords', 'tf-isf', 'position_score', 
                        'paragraph_score', 'number_citations', 'length_score', 'pos_score',
                        'ner_score', 'dist_centroid', 'articles']
    
    scores_columns = ['rouge_1', 'rouge_2', 'rouge_l', 'label', 'articles']
    
    embeddings_columns = [i for i in range(embed_dim)]
    embeddings_columns.append("article")
    

    """ if os.path.isfile('batches_{}.json'.format(batch_len)):

        print("Load Batch Files")
        with open('batches_{}.json'.format(batch_len)) as f:
            batches = json.load(f)

        batch_files = [value for key, value in batches.items()]
    else: 

        print("Creating Batch Files")
        batch_files = utils.create_batches(path_base, tam=batch_len)
        utils.save_batches(batch_files)
        

    batche_files = batch_files[initial_batch:]"""

    batche_files = os.listdir(path_base)

    print("Name section: " + name_section)
    vfunc = np.vectorize(pef.extract_features_file)
    
    print("Iniciando a extração de features...")
    
    for batch in batche_files:
        
        print(batch)

    
        pef.extract_features_batches(
            vfunc, [batch], path_base, name_section=name_section, features_columns=features_columns,
            scores_columns=scores_columns, embeddings_columns=embeddings_columns, verbose=verbose)
 

In [184]:
%load_ext autoreload
%autoreload 2

from src import preprocess
from src import extract_features
from src import tokenizer
from src import create_features_df
from src import transform_data
from src import loader
from src import utils
from src import ensemble_tree_models
from src import tunning_hyperparametrs as th
#from src import mlp_regressor
#from src import mlp_classifier
from src import summarization
from src import normalization
from src import ensemble_tree_models as classifiers
from src import utils_classification as utils_clf
from src import evaluate_classifiers as ev
from src import prepare_data
from src import display_results as dr

from joblib import Parallel, delayed
#from tensorflow.keras.utils import to_categorical
from src import pipeline_extract_features as pef
from joblib import Parallel, delayed

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [183]:
initial_batch=0
#batche_files = batch_files[initial_batch:]

path = ["../../PMC002xxxxxx_pp", "../../PMC003xxxxxx_pp", "../../PMC005xxxxxx_pp", "../../PMC006xxxxxx_pp"]

sections = ['introduction', 'materials', 'conclusion']


In [182]:
from multiprocessing import Process, Queue

l1 = Queue()
p1 = Process(
    target=main, args=(
        sections[0], initial_batch, 300,
        700, path[0], False))

l2 = Queue()
p2 = Process(target=main,  args=(
        sections[1], initial_batch, 300,
        700, path[0], False))

l3 = Queue()
p3 = Process(target=main, args=(
        sections[2], initial_batch, 300,
        700, path[0], False))

l4 = Queue()
p4 = Process(
    target=main, args=(
        sections[0], initial_batch, 300,
        700, path[1], False))

l5 = Queue()
p5 = Process(target=main,  args=(
        sections[1], initial_batch, 300,
        700, path[1], False))

l6 = Queue()
p6 = Process(target=main, args=(
        sections[2], initial_batch, 300,
        700, path[1], False))

l7 = Queue()
p7 = Process(
    target=main, args=(
        sections[0], initial_batch, 300,
        700, path[2], False))

l8 = Queue()
p8 = Process(target=main,  args=(
        sections[1], initial_batch, 300,
        700, path[2], False))

l9 = Queue()
p9 = Process(target=main, args=(
        sections[2], initial_batch, 300,
        700, path[2], False))

l10 = Queue()
p10 = Process(
    target=main, args=(
        sections[0], initial_batch, 300,
        700, path[3], False))

l11 = Queue()
p11 = Process(target=main,  args=(
        sections[1], initial_batch, 300,
        700, path[3], False))

l12 = Queue()
p12 = Process(target=main, args=(
        sections[2], initial_batch, 300,
        700, path[3], False))

l1 = Queue()
p10 = Process(
    target=main, args=(
        sections[0], initial_batch, 300,
        700, path[4], False))

l11 = Queue()
p11 = Process(target=main,  args=(
        sections[1], initial_batch, 300,
        700, path[4], False))

l12 = Queue()
p12 = Process(target=main, args=(
        sections[2], initial_batch, 300,
        700, path[4], False))

l10 = Queue()
p10 = Process(
    target=main, args=(
        sections[0], initial_batch, 300,
        700, path[5], False))

l11 = Queue()
p11 = Process(target=main,  args=(
        sections[1], initial_batch, 300,
        700, path[5], False))

l12 = Queue()
p12 = Process(target=main, args=(
        sections[2], initial_batch, 300,
        700, path[5], False))

l10 = Queue()
p10 = Process(
    target=main, args=(
        sections[0], initial_batch, 300,
        700, path[6], False))

l11 = Queue()
p11 = Process(target=main,  args=(
        sections[1], initial_batch, 300,
        700, path[6], False))

l12 = Queue()
p12 = Process(target=main, args=(
        sections[2], initial_batch, 300,
        700, path[6], False))

p1.start()   
p2.start()   
p3.start()
p4.start()   
p5.start()   
p6.start()
p7.start()   
p8.start()   
p9.start()
p10.start()   
p11.start()   
p12.start()

Name section: introductionName section: materialsName section: introductionName section: introductionName section: introductionName section: materialsName section: introductionName section: conclusionName section: conclusionName section: conclusionName section: conclusionName section: materialsName section: materialsName section: conclusionName section: materials










Iniciando a extração de features...


Iniciando a extração de features...Iniciando a extração de features...Iniciando a extração de features...Iniciando a extração de features...Iniciando a extração de features...Iniciando a extração de features...Iniciando a extração de features...Iniciando a extração de features...Iniciando a extração de features...Iniciando a extração de features...Iniciando a extração de features...Iniciando a extração de features...

Iniciando a extração de features...

Iniciando a extração de features...







PMC8248445.jsonPMC2858566.json
PMC8248445.jsonPMC8248445.json
PMC4196426.jsonPMC7654

9  0.000000  0.000000  0.000000PMC7871897.json

     rouge_1   rouge_2   rouge_l
0   0.297872  0.000000  0.276867
1   0.262295  0.034483  0.224668
2   0.181818  0.048780  0.224776
3   0.242424  0.000000  0.241702
4   0.188679  0.000000  0.165693
5   0.260870  0.058824  0.200246
6   0.117647  0.000000  0.169103
7   0.200000  0.047619  0.217542
8   0.238095  0.050000  0.197866
9   0.255319  0.045455  0.276867
10  0.507463  0.153846  0.395628
11  0.518519  0.269231  0.437216
12  0.256410  0.078947  0.268111
13  0.150000  0.000000  0.147011
14  0.000000  0.000000  0.000000
PMC5922272.json
PMC8034595.json
     rouge_1   rouge_2   rouge_l
0   0.255319  0.222222  0.320808
1   0.476190  0.100000  0.440774
2   0.375000  0.121212  0.340196
3   0.210526  0.055556  0.276496
4   0.317460  0.057143  0.286388
5   0.350515  0.084211  0.299735
6   0.250000  0.111111  0.315092
7   0.257143  0.059701  0.233777
8   0.259740  0.028571  0.213793
9   0.237288  0.051282  0.256216
10  0.222222  0.058824  0.285

50  0.000000  0.000000  0.000000
PMC7443051.json
PMC7003534.json     rouge_1   rouge_2   rouge_l
0   0.375000  0.200000  0.441717
1   0.450704  0.144928  0.348153
2   0.533333  0.279070  0.592751
3   0.425000  0.250000  0.462015
4   0.294118  0.063492  0.280684
5   0.231884  0.055556  0.220327
6   0.383562  0.197183  0.423040
7   0.257426  0.075000  0.244831
8   0.431373  0.163265  0.341626
9   0.349206  0.067797  0.295812
10  0.289855  0.096386  0.274739
11  0.148148  0.000000  0.206129
12  0.315789  0.054054  0.273006
13  0.250000  0.093023  0.237286
14  0.258824  0.025974  0.251368
15  0.297872  0.153846  0.308578
16  0.222222  0.046512  0.237286
17  0.187500  0.032258  0.237286
18  0.350877  0.181818  0.273241
19  0.105263  0.000000  0.153941
20  0.285714  0.083333  0.241702
21  0.222222  0.040000  0.235162
22  0.184211  0.046512  0.153197
23  0.333333  0.181818  0.317490
24  0.190476  0.000000  0.179367
25  0.230769  0.059701  0.233055
26  0.210526  0.093023  0.244913
27  0.328358

PMC5097894.json
     rouge_1   rouge_2   rouge_l
0   0.266667  0.085106  0.237748
1   0.250000  0.000000  0.318979
2   0.200000  0.060606  0.205926
3   0.125000  0.000000  0.176825
4   0.484848  0.322581  0.493316
5   0.250000  0.060606  0.241579
6   0.238095  0.047619  0.251751
7   0.222222  0.000000  0.190763
8   0.228571  0.000000  0.261857
9   0.388889  0.111111  0.382676
10  0.333333  0.130435  0.360324
11  0.206897  0.058824  0.269048
12  0.173913  0.000000  0.186776
13  0.150000  0.066667  0.206107
14  0.157895  0.000000  0.156704
15  0.243902  0.000000  0.201801
16  0.148148  0.000000  0.204633
17  0.162162  0.000000  0.172400
18  0.178571  0.042553  0.214581
19  0.200000  0.000000  0.187485
20  0.226415  0.000000  0.235092
21  0.326531  0.133333  0.322296
22  0.378378  0.228571  0.445113
23  0.000000  0.000000  0.000000
PMC3714332.json
PMC8046841.json
    rouge_1   rouge_2   rouge_l
0  0.560000  0.375000  0.618550
1  0.333333  0.100000  0.352055
2  0.400000  0.172414  0.298880

16  0.000000  0.000000  0.000000
PMC5930123.json

PMC4056449.json
PMC7686265.json
PMC8217134.json
PMC4225219.json
PMC6864273.json
PMC7610609.json
PMC2630050.json
PMC5561521.json
PMC3197902.json
PMC5222617.json
PMC3119467.json
PMC8202570.json
PMC8026481.json
PMC3281556.json
PMC6374028.json
PMC8075319.json
PMC8363189.json
PMC4161035.json
PMC2941709.json
PMC5903999.json
PMC3772646.json
PMC7005901.json
PMC6863078.json
PMC8202570.json
     rouge_1   rouge_2   rouge_l
0   0.244898  0.042553  0.220898
1   0.117647  0.000000  0.168068
2   0.360000  0.166667  0.263448
3   0.153846  0.054054  0.150248
4   0.333333  0.105263  0.348866
5   0.210526  0.054054  0.273603
6   0.206897  0.047059  0.220189
7   0.133333  0.046512  0.187332
8   0.246914  0.126582  0.285779
9   0.274510  0.081633  0.299663
10  0.250000  0.076923  0.234246
11  0.000000  0.000000  0.000000
     rouge_1   rouge_2   rouge_l
0   0.315789  0.123457  0.305467
1   0.329670  0.179775  0.329562
2   0.363636  0.115942  0.304186
3   0

...

55  0.000000  0.000000  0.000000
PMC7571257.json
PMC3763952.json
PMC5955819.json
     rouge_1   rouge_2   rouge_l
0   0.193548  0.000000  0.256009
1   0.350000  0.136364  0.371527
2   0.280000  0.125000  0.304479
3   0.342857  0.114286  0.310140
4   0.243902  0.000000  0.256301
5   0.230769  0.040000  0.253541
6   0.271186  0.000000  0.294656
7   0.382979  0.177778  0.275798
8   0.181818  0.000000  0.173674
9   0.300000  0.000000  0.263210
10  0.280000  0.038462  0.217371
11  0.153846  0.000000  0.151104
12  0.277778  0.117647  0.286526
13  0.263158  0.000000  0.262267
14  0.214286  0.000000  0.214800
15  0.166667  0.051282  0.203839
16  0.150000  0.000000  0.183376
17  0.400000  0.071429  0.332797
18  0.125000  0.000000  0.182217
19  0.000000  0.000000  0.000000
     rouge_1   rouge_2   rouge_l
0   0.333333  0.117647  0.315593
1   0.222222  0.000000  0.225104
2   0.363636  0.064516  0.205926
3   0.312500  0.000000  0.204633
4   0.187500  0.000000  0.230017
5   0.533333  0.357143  0.33

13  0.000000  0.000000  0.000000
PMC3357206.json
PMC6688897.json
PMC4049341.json
PMC7485011.json
PMC2819533.json
PMC8023688.json
    rouge_1   rouge_2   rouge_l
0  0.329897  0.056338  0.268833
1  0.375000  0.233010  0.347488
2  0.000000  0.000000  0.000000
PMC5112131.json
PMC8335750.json
    rouge_1   rouge_2   rouge_l
0  0.258065  0.000000  0.216830
1  0.418605  0.195122  0.484009
2  0.000000  0.000000  0.000000
3  0.769231  0.594595  0.804535
4  0.400000  0.181818  0.409844
5  0.372881  0.210526  0.373535
6  0.367347  0.160000  0.266917
7  0.363636  0.096774  0.344605
8  0.466667  0.166667  0.371720
9  0.000000  0.000000  0.000000
PMC6613779.json
PMC7979517.json
PMC4176821.json
PMC3766324.json
     rouge_1   rouge_2   rouge_l
0   0.190476  0.000000  0.251151
1   0.044444  0.000000  0.076144
2   0.131148  0.000000  0.144775
3   0.071429  0.000000  0.110930
4   0.153846  0.000000  0.210960
5   0.088889  0.000000  0.118150
6   0.081633  0.042553  0.125015
7   0.074074  0.000000  0.11470

3  0.000000     0.00  0.000000
PMC7064371.json
PMC2777536.json
PMC8248447.json
     rouge_1   rouge_2   rouge_l
0   0.434783  0.272727  0.500121
1   0.405405  0.190476  0.393303
2   0.437500  0.290323  0.503820
3   0.716418  0.523077  0.731258
4   0.307692  0.054054  0.267219
5   0.812500  0.666667  0.787694
6   0.500000  0.303030  0.466024
7   0.280702  0.072727  0.273100
8   0.411765  0.064516  0.307025
9   0.255319  0.054054  0.275588
10  0.326531  0.085106  0.272220
11  0.279070  0.048780  0.246692
12  0.369231  0.190476  0.343281
13  0.418605  0.200000  0.417535
14  0.266667  0.117647  0.286335
15  0.325581  0.050000  0.297108
16  0.250000  0.100000  0.228877
17  0.262626  0.027027  0.208170
18  0.352941  0.062500  0.360749
19  0.190476  0.040816  0.217178
20  0.215385  0.074074  0.200615
21  0.418605  0.146341  0.345859
22  0.200000  0.051282  0.203041
23  0.229885  0.090909  0.245309
24  0.309859  0.156863  0.332197
25  0.250000  0.085714  0.230250
26  0.300000  0.071429  0.2707

20  0.000000  0.000000  0.000000
PMC8086895.json
PMC6053672.json
PMC6121733.json
PMC7058493.json
PMC2693286.json
PMC7061088.json
PMC8117170.json
PMC7461716.json
PMC2875279.json
     rouge_1   rouge_2   rouge_l
0   0.513761  0.168224  0.452879
1   0.185567  0.037037  0.171816
2   0.225806  0.066667  0.293089
3   0.303030  0.064516  0.307182
4   0.280702  0.125000  0.311979
5   0.525424  0.206897  0.489275
6   0.363636  0.213333  0.381880
7   0.190476  0.000000  0.197866
8   0.325581  0.054054  0.297108
9   0.198198  0.028986  0.220047
10  0.218182  0.043011  0.200689
11  0.245614  0.041667  0.273241
12  0.366667  0.172414  0.370935
13  0.269231  0.040000  0.253148
14  0.176471  0.071429  0.186602
15  0.071429  0.000000  0.115567
16  0.304348  0.136364  0.283469
17  0.230769  0.050000  0.253113
18  0.200000  0.071429  0.186602
19  0.210526  0.036364  0.196601
20  0.269663  0.062992  0.217123
21  0.200000  0.022727  0.189259
22  0.234043  0.039216  0.203741
23  0.545455  0.190476  0.43097

PMC7402636.json
    rouge_1   rouge_2   rouge_l
0  0.293333  0.082192  0.291605
1  0.390805  0.141176  0.365900
2  0.306452  0.098361  0.237415
3  0.000000  0.000000  0.000000

In [29]:
import sys
sys.path.insert(1, 'src')

import glob, os
import pandas as pd
import json
import spacy
import nltk
import numpy as np
import json

import seaborn as sns
import pickle
from pathlib import Path


from bs4 import BeautifulSoup
from pysbd.utils import PySBDFactory
import math

from sumeval.metrics.rouge import RougeCalculator
rouge = RougeCalculator(stopwords=True, lang="en")

from timeit import default_timer as timer  

from src import preprocess
from src import extract_features
from src import tokenizer
from src import create_features_df
from src import transform_data
from src import loader
from src import utils
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import random

import warnings
warnings.filterwarnings("ignore")

nlp_sm = spacy.load('en_core_web_sm')
nlp_md = spacy.load('en_core_web_md')

In [187]:
batche_files = os.listdir('../result/materials')

In [188]:
batche_files

In [186]:
len(batche_files)

In [176]:
df = utils.load_batches(
            ['PMC2813960.json'], path[0], 'introduction')

In [177]:
section = df['texts'][0]
reference=df['abstract'][0]
keywords=df['keywords'][0]

In [178]:
%load_ext autoreload
%autoreload 2

from src import preprocess
from src import extract_features
from src import tokenizer
from src import create_features_df
from src import transform_data
from src import loader
from src import utils
from src import ensemble_tree_models
from src import tunning_hyperparametrs as th
#from src import mlp_regressor
#from src import mlp_classifier
from src import summarization
from src import normalization
from src import ensemble_tree_models as classifiers
from src import utils_classification as utils_clf
from src import evaluate_classifiers as ev
from src import prepare_data
from src import display_results as dr

from joblib import Parallel, delayed
#from tensorflow.keras.utils import to_categorical
from src import pipeline_extract_features as pef
from joblib import Parallel, delayed

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [179]:
xml = preprocess.format_xml(str(section))
text = preprocess.format_text(str(section), post_processing=False)
reference = preprocess.format_text(str(reference), post_processing=True)


bibs = extract_features.get_citations(xml)
text = preprocess.replace_bib(text, bibs)
text = preprocess.format_text(text, post_processing=True)

soup = BeautifulSoup(text)
text = soup.get_text()

sentences = tokenizer.split_sentences([text])
sentences = list(map(str, sentences[0]))
sentences = preprocess.format_sentences(sentences)

features_df= create_features_df.main(sentences, xml, keywords, nlp_sm, nlp_md)


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [180]:
sentences

["b'  '"]

In [83]:
for key in features.keys():
    print("key: {}, len: {}".format(key, len(features[key])))

key: pos_score, len: 16
key: ner_score, len: 16
key: position_score, len: 16
key: number_citations, len: 16
key: paragraph_score, len: 16
key: length_score, len: 16
key: onegram, len: 16
key: two_gram, len: 16
key: three_gram, len: 16
key: count_article_keywords, len: 16
key: cluster_df, len: 16
key: result_tfisf, len: 16


In [102]:
features_df

,pos_score,ner_score,position_score,number_citations,paragraph_score,length_score,onegram,two_gram,three_gram,count_article_keywords,cluster_df,result_tfisf,sentences,sentences_text,sentences_lsa,sentences_lex
0,9,1,1.0000,0,1,19,1,0,0,0,0.944572,0.015016,b' Proteins carry out a variety of complicated...,0.001821,0.062500,0.062500
1,16,0,1.0000,0,0,24,8,1,0,0,0.954411,0.021094,Many of these important activities utilize the...,0.001925,0.056250,0.056250
2,24,0,0.9375,0,0,42,12,1,0,0,0.975672,0.024137,In contrast to small molecule reactions for wh...,0.003812,0.073958,0.073958
3,38,2,0.8750,0,0,68,7,1,0,0,0.976150,0.031974,Much recent work has focused on this issue (fo...,0.004431,0.070833,0.070833
4,19,0,0.8125,0,0,29,9,1,0,0,0.966898,0.022983,A remaining challenge in linking protein confo...,0.003345,0.084375,0.084375
5,24,0,0.7500,0,1,38,10,0,0,0,0.968752,0.024235,To implement a strategy to characterize transi...,0.003749,0.068750,0.068750
6,4,0,0.6875,0,0,9,0,0,0,0,0.848580,0.010677,What are the inter conversion rates between st...,0.001297,0.052083,0.052083
7,8,0,0.6250,0,0,13,2,0,0,0,0.873898,0.012946,How do ligand binding or interactions with oth...,0.001532,0.052083,0.052083
8,4,0,0.5625,0,0,8,3,0,0,0,0.000000,0.010640,What functional roles do these motions play?,0.001151,0.083333,0.083333
9,15,0,0.5000,0,0,27,6,0,0,0,0.952860,0.018405,It is important to note that only when the ran...,0.003282,0.056250,0.056250


In [97]:
df = pd.DataFrame(features)
df['sentences'] = sentences

summ = pd.DataFrame(summ).reset_index()
summ=summ.rename(columns={'index': 'sentences'})

df = df.merge(summ, on='sentences')

In [98]:
df

,pos_score,ner_score,position_score,number_citations,paragraph_score,length_score,onegram,two_gram,three_gram,count_article_keywords,cluster_df,result_tfisf,sentences,sentences_text,sentences_lsa,sentences_lex
0,9,1,1.0000,0,1,19,1,0,0,0,0.944572,0.015016,b' Proteins carry out a variety of complicated...,0.001821,0.062500,0.062500
1,16,0,1.0000,0,0,24,8,1,0,0,0.954411,0.021094,Many of these important activities utilize the...,0.001925,0.056250,0.056250
2,24,0,0.9375,0,0,42,12,1,0,0,0.975672,0.024137,In contrast to small molecule reactions for wh...,0.003812,0.073958,0.073958
3,38,2,0.8750,0,0,68,7,1,0,0,0.976150,0.031974,Much recent work has focused on this issue (fo...,0.004431,0.070833,0.070833
4,19,0,0.8125,0,0,29,9,1,0,0,0.966898,0.022983,A remaining challenge in linking protein confo...,0.003345,0.084375,0.084375
5,24,0,0.7500,0,1,38,10,0,0,0,0.968752,0.024235,To implement a strategy to characterize transi...,0.003749,0.068750,0.068750
6,4,0,0.6875,0,0,9,0,0,0,0,0.848580,0.010677,What are the inter conversion rates between st...,0.001297,0.052083,0.052083
7,8,0,0.6250,0,0,13,2,0,0,0,0.873898,0.012946,How do ligand binding or interactions with oth...,0.001532,0.052083,0.052083
8,4,0,0.5625,0,0,8,3,0,0,0,0.000000,0.010640,What functional roles do these motions play?,0.001151,0.083333,0.083333
9,15,0,0.5000,0,0,27,6,0,0,0,0.952860,0.018405,It is important to note that only when the ran...,0.003282,0.056250,0.056250


In [92]:
summ = pd.DataFrame(summ).reset_index()
summ=summ.rename(columns={'index': 'sentences'})

In [161]:
import rouge


def prepare_results(m, p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(m, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


for aggregator in ['Avg', 'Best', 'Individual']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=False,
                           apply_best=True,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)


    hypothesis_1 = "King Norodom Sihanouk has declined requests to chair a summit of Cambodia 's top political leaders , saying the meeting would not bring any progress in deadlocked negotiations to form a government .\nGovernment and opposition parties have asked King Norodom Sihanouk to host a summit meeting after a series of post-election negotiations between the two opposition groups and Hun Sen 's party to form a new government failed .\nHun Sen 's ruling party narrowly won a majority in elections in July , but the opposition _ claiming widespread intimidation and fraud _ has denied Hun Sen the two-thirds vote in parliament required to approve the next government .\n"
    references_1 = ["Prospects were dim for resolution of the political crisis in Cambodia in October 1998.\nPrime Minister Hun Sen insisted that talks take place in Cambodia while opposition leaders Ranariddh and Sam Rainsy, fearing arrest at home, wanted them abroad.\nKing Sihanouk declined to chair talks in either place.\nA U.S. House resolution criticized Hun Sen's regime while the opposition tried to cut off his access to loans.\nBut in November the King announced a coalition government with Hun Sen heading the executive and Ranariddh leading the parliament.\nLeft out, Sam Rainsy sought the King's assurance of Hun Sen's promise of safety and freedom for all politicians.",
                    "Cambodian prime minister Hun Sen rejects demands of 2 opposition parties for talks in Beijing after failing to win a 2/3 majority in recent elections.\nSihanouk refuses to host talks in Beijing.\nOpposition parties ask the Asian Development Bank to stop loans to Hun Sen's government.\nCCP defends Hun Sen to the US Senate.\nFUNCINPEC refuses to share the presidency.\nHun Sen and Ranariddh eventually form a coalition at summit convened by Sihanouk.\nHun Sen remains prime minister, Ranariddh is president of the national assembly, and a new senate will be formed.\nOpposition leader Rainsy left out.\nHe seeks strong assurance of safety should he return to Cambodia.\n",
                    ]

    hypothesis_2 = "China 's government said Thursday that two prominent dissidents arrested this week are suspected of endangering national security _ the clearest sign yet Chinese leaders plan to quash a would-be opposition party .\nOne leader of a suppressed new political party will be tried on Dec. 17 on a charge of colluding with foreign enemies of China '' to incite the subversion of state power , '' according to court documents given to his wife on Monday .\nWith attorneys locked up , harassed or plain scared , two prominent dissidents will defend themselves against charges of subversion Thursday in China 's highest-profile dissident trials in two years .\n"
    references_2 = "Hurricane Mitch, category 5 hurricane, brought widespread death and destruction to Central American.\nEspecially hard hit was Honduras where an estimated 6,076 people lost their lives.\nThe hurricane, which lingered off the coast of Honduras for 3 days before moving off, flooded large areas, destroying crops and property.\nThe U.S. and European Union were joined by Pope John Paul II in a call for money and workers to help the stricken area.\nPresident Clinton sent Tipper Gore, wife of Vice President Gore to the area to deliver much needed supplies to the area, demonstrating U.S. commitment to the recovery of the region.\n"

    all_hypothesis = [hypothesis_1]
    all_references = [references_1, references_2]

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(metric,results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(metric, results['p'], results['r'], results['f']))
    print()

Evaluation with Avg


ValueError: 'hyps' and 'refs' do not have the same length

In [ ]:
def format_metrics(df, scores, name_model, metrics):
    
    for metric in metrics:
        df['{}_{}'.format(name_model, metric)]= [score['f'][0] for score in scores[metric]]
        
    return df

In [151]:
r1, r2, rl =  scores['rouge-1'][0]['f'][0], scores['rouge-2'][0]['f'], scores['rouge-l'][0]['f']

In [152]:
r1

0.37